In [1]:
%load_ext autoreload
%autoreload 2

## evolver

In [ ]:
import torch
from csg import Evolver, CSG, CSGTreeDataset
from torch.utils.data import DataLoader

csg = CSG()

model = Evolver(
    d_model=256,
    dim_feedforward=256*4,
    nhead=16,
    dropout=0.1,
    layer_norm_eps=1e-5,
    encoder_layers=4,
    decoder_layers=4,
    vocab_size=csg.vocab_size,
    max_len=256,
    pad_token_id=csg.tok_to_id['PAD'],
    bos_token_id=csg.tok_to_id['BOS'],
    eos_token_id=csg.tok_to_id['EOS'],
    root_id=csg.tok_to_id['s'],
    csg=csg,
    name=None
)

# model.load_state_dict(torch.load('csg_evolver_256d_4enc_4dec-20240924_123548_45000.pt', map_location='cpu')['model'])
model.load_state_dict(torch.load('csg_evolver_256d_4enc_4dec-20240925_232201_45000.pt', map_location='cpu')['model'])

dataset = CSGTreeDataset()
loader = DataLoader(dataset, batch_size=5, collate_fn=dataset.collate_fn)

In [ ]:
import matplotlib.pyplot as plt

batch = next(iter(loader))

print(batch['programs'][0])

plt.imshow(batch['imgs'][0].squeeze(), cmap='gray')
plt.show()

In [ ]:
# model.eval()
model.train()

traj, output_ids = model._generate_unbatched(batch['imgs'][0], max_depth=20, max_steps=100)

# output_ids = model.generate(batch['imgs'], max_depth=20, max_steps=100)

In [ ]:
for t in traj:
    print(csg.detokenize_tensor(t))

# csg.detokenize_tensor(output_ids)

# from csg import evaluate

In [ ]:
csg.render('( Circle ( Num 3 ) ( Num 2 ) ( Num 8 ) )')

## decoder only

In [ ]:
import torch
from torch.utils.data import DataLoader
from csg import DecoderOnlyTransformer, CSGDataset, CSG

csg = CSG()

model = DecoderOnlyTransformer(
    d_model=256,
    dim_feedforward=256*4,
    nhead=16,
    dropout=0.1,
    layer_norm_eps=1e-5,
    decoder_layers=8,
    vocab_size=csg.vocab_size,
    max_len=256,
    pad_token_id=csg.tok_to_id['PAD'],
    bos_token_id=csg.tok_to_id['BOS'],
    eos_token_id=csg.tok_to_id['EOS'],
    name=None
)

model.load_state_dict(torch.load('csg_decoder_only_256d_0enc_8dec-20240923_011743_100000.pt', map_location='cpu')['model'])

dataset = CSGDataset()
loader = DataLoader(dataset, batch_size=5, collate_fn=dataset.collate_fn)

In [ ]:
import matplotlib.pyplot as plt

batch = next(iter(loader))
img = batch['imgs'][0]
print(batch['programs'][0])

plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
program = batch['programs'][0]
tree = csg.parse(program)

for i in range(10):
    print(csg.expand(tree, max_depth=i))

In [ ]:
output_ids = model.generate(batch['imgs'][0].unsqueeze(0))

programs = csg.detokenize_tensor(output_ids, skip_special_tokens=True)
print(programs[0])

csg.render(programs[0])

In [ ]:
print(programs[0])
tree = csg.parse(programs[0])
for i in range(10): print(csg.expand(tree, max_depth=i))

In [ ]:
!python csg.py --config=configs/csg/ar.json --local

In [ ]:
!python csg.py --config=configs/csg/evo.json --local